In [1]:
import pandas as pd
import random

# First generate a random dataset with 3 features: age, gender and daytime, each point is either click or no click.

data = pd.DataFrame()

num_data_points = 1000000

ages = [random.randint(18, 60) for _ in range(num_data_points)]
is_fem = [random.choice([0,1]) for _ in range(num_data_points)]
is_other = [random.choice([0,1]) for _ in range(num_data_points)]
is_male = [random.choice([0,1]) for _ in range(num_data_points)]

is_morning = [random.choice([0,1]) for _ in range(num_data_points)]
is_noon = [random.choice([0,1]) for _ in range(num_data_points)]
is_evening = [random.choice([0,1]) for _ in range(num_data_points)]
is_night = [random.choice([0,1]) for _ in range(num_data_points)]

clicks = [random.choice([0, 1]) for _ in range(num_data_points)]

data['Age'] = ages
data['Is_Female'] = is_fem
data['Is_Male'] = is_male
data['Is_Other'] = is_other
data['Is_Morning'] = is_morning
data['Is_Noon'] = is_noon
data['Is_Evening'] = is_evening
data['Is_Night'] = is_night
data['Click'] = clicks

print(data.head())

   Age  Is_Female  Is_Male  Is_Other  Is_Morning  Is_Noon  Is_Evening  \
0   42          1        1         0           0        1           0   
1   32          0        0         1           0        0           0   
2   18          1        1         0           1        0           1   
3   27          0        1         1           0        1           1   
4   32          1        0         1           0        0           1   

   Is_Night  Click  
0         0      0  
1         0      0  
2         1      0  
3         1      1  
4         0      1  


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Separate features (X) and labels (y)
X = data[['Age', 'Is_Female', 'Is_Male', 'Is_Other', 'Is_Morning', 'Is_Noon', 'Is_Evening', 'Is_Night']]
y = data['Click']

In [3]:
import lightgbm as lgb

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = lgb.LGBMClassifier()

clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Convert predicted probabilities to binary class labels (0 or 1)
y_pred_binary = [1 if p >= 0.5 else 0 for p in y_pred]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred_binary)
report = classification_report(y_test, y_pred_binary)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

[LightGBM] [Info] Number of positive: 399918, number of negative: 400082
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007776 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 58
[LightGBM] [Info] Number of data points in the train set: 800000, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499897 -> initscore=-0.000410
[LightGBM] [Info] Start training from score -0.000410
Accuracy: 0.50023
Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.51      0.50    100071
           1       0.50      0.49      0.50     99929

    accuracy                           0.50    200000
   macro avg       0.50      0.50      0.50    200000
weighted avg       0.50      0.50      0.50    200000



In [4]:
clf.feature_name_

['Age',
 'Is_Female',
 'Is_Male',
 'Is_Other',
 'Is_Morning',
 'Is_Noon',
 'Is_Evening',
 'Is_Night']

In [5]:
import onnxmltools
from onnxmltools.convert import convert_lightgbm
from skl2onnx.common.data_types import FloatTensorType

features=str(clf.feature_name_)
initial_types = [('input', FloatTensorType([1, X_train.shape[1]]))]
onnx_model = convert_lightgbm(clf, initial_types=initial_types, zipmap=False)



ImportError: cannot import name 'FEATURE_IMPORTANCE_TYPE_MAPPER' from 'lightgbm.basic' (/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/lightgbm/basic.py)

In [45]:
# Save as protobuf
onnxmltools.utils.save_model(onnx_model, 'test.onnx')

NameError: name 'onnx_model' is not defined